In [13]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.analysis import *
from whoosh.qparser import *
from whoosh import scoring
from whoosh import *
from whoosh.writing import AsyncWriter
import csv
import time
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from Eval_metrics import * #my script .py with the implementation of the eval metrics

import pylab 
from matplotlib import colors as mcolors
import random

In [2]:
''' Define a function to convert the html file into csv'''
def converter():
    #path to the html files for the Cranfield_DATASET
    path_Cranfield = os.getcwd()+ "\part_1\part_1_1\Cranfield_DATASET\DOCUMENTS\_" 
    #path to the html files for the Time_DATASET
    path_Time = os.getcwd()+ "\part_1\part_1_1\Time_DATASET\DOCUMENTS\_" 
    #initialization of the dataframe for the crainfield csv   
    cranfield_df=pd.DataFrame(columns=['ID','title','body']) 
    #initialization of the dataframe for the time csv
    time_df=pd.DataFrame(columns=['ID','body']) 
    # for each doc in the folder Cranfield_DATASET
    for i in range(1,1401): 
        #define the file name
        filename1=path_Cranfield+'_'*5+str(i)+'.html' 
        #open the file in reading mode
        with open(filename1) as f:
            content = f.read() 
        #use a html parser to pick the content of the file 
        soup = BeautifulSoup(content, 'html.parser')
        # save the title 
        title=soup.title.string 
        # save the body
        body=soup.body.string 
        #store into the dataframe the content. Each row of the document df is a .html file
        cranfield_df=cranfield_df.append({'ID':i,'title':title,'body':body},ignore_index=True) 
    
    #for each doc in the folder Time_DATASET
    for j in range(1,423): 
        #define the file name
        filename2=path_Time+'_'*5+str(j)+'.html' 
        #open the file in reading mode
        with open(filename2) as f:
            content = f.read() 
        #use a html parser to pick the content of the file 
        soup = BeautifulSoup(content, 'html.parser')
        # save the body
        body=soup.body.string 
        #store into the dataframe the content. Each row of the document df is a .html file
        time_df=time_df.append({'ID':j,'body':body},ignore_index=True) 
    return cranfield_df,time_df


# save the doc from Cranfield dataset into csv file format
doc_Cran_converted = converter()[0]
doc_Cran_converted.to_csv(os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET\doc_to_index.csv")

# save the doc from Time dataset into csv file format
doc_Time_converted = converter()[1]
doc_Time_converted.to_csv(os.getcwd()+"\part_1\part_1_1\Time_DATASET\Time_to_index.csv")

In [3]:
'''Function to create the dictionaries from the se and gt'''
def create_dict(sr,gt,Q):
    dict_se= {} 
    dict_gt= {} 
    for i in Q:
        #key=Query_id,value=list of document ids from SE result
        dict_se[i]=list(sr[sr['Query_id']==i]['Doc_ID'])

    for i in Q:
        #key=Query_id,value=list of relevant document ids) from ground truth
        dict_gt[i]=list(gt[gt['Query_id']==i]['Relevant_Doc_id'])
    return dict_se,dict_gt

In [4]:
'''DEFINE THE FIRST PART OF THE SOFTWARE FOR THE SE''' 
def sw_1(analyzer,filename,ds):
    if ds == 0:
        # creating schema with fields id, title and content
        schema = Schema(id=ID(stored=True),title=TEXT(stored=False, analyzer=analyzer),\
                    content=TEXT(stored=False, analyzer=analyzer))
        directory_containing_the_index = os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET" 
        # create an empty-index according to the just defined schema in the directory where csv file is
        ix = create_in(directory_containing_the_index, schema) 
        # open the index file
        ix = index.open_dir(directory_containing_the_index) #open the index file 
        writer =  AsyncWriter(ix) #define a writer object to add content to the fields
        # fill the index:
        ALL_DOCUMENTS_file_name = filename #path of the file 
        # open the file
        in_file = open(ALL_DOCUMENTS_file_name, "r", encoding='latin1')
        # read it as csv
        csv_reader = csv.reader(in_file, delimiter=',')  
        # to skip the header: first line contains the name of each field.
        csv_reader.__next__()
        # for each row in the 'doc_to_index' file 
        for record in csv_reader: 
            id0 = record[1] # extract the id doc
            title = record[2] # extract the title
            content = record[3] # extract the body
            # add this new doc into the inverted index according to the schema
            writer.add_document(id=id0, content=title+' '+content)
        # commit all the operations on the file
        writer.commit()
        # close the file
        in_file.close()
    else:
        # creating schema with fields id and content
        schema1 = Schema(id=ID(stored=True),content=TEXT(stored=False, analyzer=analyzer))
        directory_containing_the_index = os.getcwd()+"\part_1\part_1_1\Time_DATASET" 
        # create an empty-index according to the just defined schema in the directory where csv file is
        ix1 = create_in(directory_containing_the_index, schema1) 
        # open the index file
        ix1 = index.open_dir(directory_containing_the_index) #open the index file 
        writer1 =  AsyncWriter(ix) #define a writer object to add content to the fields
        # fill the index:
        ALL_DOCUMENTS_file_name1 = filename #path of the file 
        # open the file
        in_file = open(ALL_DOCUMENTS_file_name, "r", encoding='latin1')
        # read it as csv
        csv_reader1 = csv.reader(in_file, delimiter=',')  
        # to skip the header: first line contains the name of each field.
        csv_reader1.__next__()
        # for each row in the 'doc_to_index' file 
        for record in csv_reader1: 
            id1 = record[1] # extract the id doc
            content = record[2] # extract the body
            # add this new doc into the inverted index according to the schema
            writer.add_document(id=id1, content=' '+content)
        # commit all the operations on the file
        writer1.commit()
        # close the file
        in_file1.close()

In [5]:
''' DEFINE THE QUERY ENGINE (THE SECOND PART OF THE SOFTWARE FOR THE SE)'''
'''
Method that given the input query and given the specific SE configuration returns the results of the search
input:
    analyzer - selected text analyzer from the whoosh library
    score_fun - selected scoring function from the whoosh library
    input_query - query that's being used for evaluation
    max_number_of_results - maximal number of results that should be retrieved which are equal to the number of relevant documents related to that specific query (which we need for calculating R-precision)
output: answer - dataframe with results of the given SE given the query; columns of dataframe: ["Rank" , "Doc_ID" , "Score"]
''' 
def sw_2(analyzer,score_fun,input_query,max_number_of_results,ds):
    if ds == 0:
        directory_containing_the_index = os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET" 
        # thanks to the ix we can retreive doc of interest for the given SE configurations
        ix = index.open_dir(directory_containing_the_index) 
        # define a QueryParser for parsing the input_query
        qp = QueryParser("content", ix.schema)
        # apply it on the given query
        parsed_query = qp.parse(input_query) 
        # create a Searcher for the Index with the given scoring function 
        searcher = ix.searcher(weighting=score_fun) 
        # store results of the query and limiting max number of results
        results = searcher.search(parsed_query,limit=max_number_of_results) 
        # define a dataframe to store the results 
        result=pd.DataFrame() 
        row=pd.DataFrame()
        for hit in results:
            row=pd.DataFrame([str(hit.rank) , int(hit['id']), str(hit.score)]).T
            result=result.append(row)
        result.columns=["Rank" , "Doc_ID" , "Score"] 
        # the column 'score' contains the values of the scoring function, that we use for having in a quantitative way the relevance of a doc for a particulare query
        searcher.close()
        return result
    else:
        directory_containing_the_index = os.getcwd()+"\part_1\part_1_1\Time_DATASET" 
        # thanks to the ix we can retreive doc of interest for the given SE configurations
        ix = index.open_dir(directory_containing_the_index) 
        # define a QueryParser for parsing the input_query
        qp = QueryParser("content", ix.schema)
        # apply it on the given query
        parsed_query = qp.parse(input_query) 
        # create a Searcher for the Index with the given scoring function 
        searcher = ix.searcher(weighting=score_fun) 
        # store results of the query and limiting max number of results
        results = searcher.search(parsed_query,limit=max_number_of_results) 
        # define a dataframe to store the results 
        result1=pd.DataFrame() 
        row=pd.DataFrame()
        for hit in results:
            row=pd.DataFrame([str(hit.rank) , int(hit['id']), str(hit.score)]).T
            result=result.append(row)
        result1.columns=["Rank" , "Doc_ID" , "Score"] 
        # the column 'score' contains the values of the scoring function, that we use for having in a quantitative way the relevance of a doc for a particulare query
        searcher.close()
        return result1        

In [6]:
'''
Method that given the specific SE configuration(analyzer,score_fun)
executes and returns the results for ALL the queries 
input:
    analyzer - selected text analyzer from the whoosh library
    score_fun - selected scoring function from the whoosh library
output: result - dataframe with the results of the given SE for ALL the queries; columns of df: ["Rank" , "Doc_ID" , "Score"]
''' 
def executor(analyzer,score_fun,ds):
    if ds == 0:
        result=pd.DataFrame() # dataframe with the results of the given SE for ALL the queries; 
        tmp=pd.DataFrame() #tmp dataframe 
        # open the file with all the queries
        Queries_file=os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET\cran_Queries.tsv"
        Queries=pd.read_csv(Queries_file,sep='\t')
        # open the file of the GT
        gt=pd.read_csv(os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET\cran_Ground_Truth.tsv", sep='\t') 
        #define a list with the unique query ids

        Q=list(gt['Query_id'].unique()) 
        dq={} #key=Query_id, value=number of relevant documents related to that query_id
        for i in Q: # for each query_id
            dq[i]=len(list(gt[gt['Query_id']==i]['Relevant_Doc_id']))

        file_toindex=os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET\doc_to_index.csv"
        # invoke the function to create the schema and to store the index file based on the retrieved 'doc_to_index.csv' file  
        sw_1(analyzer,file_toindex,0)
        # for each index in the query set
        for i in Q:
            # store the number of relevant documents related to the specific input query
            max_number_of_results_1q=dq[i] #count_of_vals(dq,i)
            if max_number_of_results_1q==0:
                max_number_of_results_1q=1
            # invoke the function that,given the input query and given the specific SE configuration,
            # returns the results of the search and store it into a tmp dataframe
            tmp=sw_2(analyzer,score_fun,list(Queries[Queries['Query_ID']==i]['Query'])[0],max_number_of_results_1q,0)
            tmp['Query_id']=i
            result=result.append(tmp) #then for each query add it to the result dataframe 
        return result
    else:
        result1=pd.DataFrame() # dataframe with the results of the given SE for ALL the queries; 
        tmp1=pd.DataFrame() #tmp dataframe 
        # open the file with all the queries
        Queries_file1=os.getcwd()+"\part_1\part_1_1\Time_DATASET\time_Queries.tsv"
        Queries1=pd.read_csv(Queries_file1,sep='\t')
        # open the file of the GT
        gt1=pd.read_csv(os.getcwd()+"\part_1\part_1_1\Time_DATASET\time_Ground_Truth.tsv", sep='\t') 
        #define a list with the unique query ids

        Q1=list(gt1['Query_id'].unique()) 
        dq1={} #key=Query_id, value=number of relevant documents related to that query_id
        for i in Q1: # for each query_id
            dq1[i]=len(list(gt1[gt1['Query_id']==i]['Relevant_Doc_id']))

        file_toindex1=os.getcwd()+"\part_1\part_1_1\Time_DATASET\Time_to_index.csv"
        # invoke the function to create the schema and to store the index file based on the retrieved 'doc_to_index.csv' file  
        sw_1(analyzer,file_toindex,1)
        # for each index in the query set
        for i in Q1:
            # store the number of relevant documents related to the specific input query
            max_number_of_results_1q=dq1[i] #count_of_vals(dq,i)
            if max_number_of_results_1q==0:
                max_number_of_results_1q=1
            # invoke the function that,given the input query and given the specific SE configuration,
            # returns the results of the search and store it into a tmp dataframe
            tmp1=sw_2(analyzer,score_fun,list(Queries[Queries['Query_ID']==i]['Query'])[0],max_number_of_results_1q,1)
            tmp1['Query_id']=i
            result1=result1.append(tmp) #then for each query add it to the result dataframe 
        return result1

In [29]:
def search_engine(ds):
    if ds == 0:
        # open the ground truth
        gt=pd.read_csv(os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET\cran_Ground_Truth.tsv", sep='\t')
        list_mrr=[] # to store the MRR values for each SE configuration 
        # define the scoring functions TO CHANGE 
        score_functions = [scoring.TF_IDF(),scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)]
        # define the text analyzers
        analyzers = [StemmingAnalyzer(),RegexAnalyzer(),FancyAnalyzer(),LanguageAnalyzer('en')]
        #combinations for every chosen analyzer with every chosen scoring function
        num_analyzers = len(analyzers)
        num_score_fun = len(score_functions)
        i=1
        sel_ana=['StemmingAnalyzer()','RegexAnalyzer()','FancyAnalyzer()','LanguageAnalyzer()']#text which will be used for graph and for mrr table
        scor_func=[' TF_IDF',' BM25F']
        for x in range(num_analyzers):
            for y in range(num_score_fun):
                print(sel_ana[x]+scor_func[y])
                # execute queries with the chosen configuration
                sr_1=executor(analyzers[x],score_functions[y],0) 
                #save results of the search engine
                sr_1.to_csv(os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET"+str(i)+".csv",index=False) 
                #compute the MRR 
                list_mrr.append((sel_ana[x]+scor_func[y],MRR(sr_1,gt))) 
                i+=1
        # save into a table with MRR evaluation for every search engine configuration 
        mrrs=pd.DataFrame(list_mrr)
        mrrs.to_csv(os.getcwd()+"\part_1\part_1_1\Cranfield_DATASET\mrr.csv", index=False) #store MRR table
    else:
        # open the ground truth
        #ERRORE NEL LEGGERE IL FILE -> CHECK TUTTA LA PARTE DEL TIME DATASET
        gt1=pd.read_csv(os.getcwd()+"\part_1\part_1_1\Time_DATASET\time_Ground_Truth.tsv",sep='\t')
        list_mrr1=[] # to store the MRR values for each SE configuration 
        # define the scoring functions TO CHANGE 
        score_functions = [scoring.TF_IDF(),scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)]
        # define the text analyzers
        analyzers = [StemmingAnalyzer(),RegexAnalyzer(),FancyAnalyzer(),LanguageAnalyzer('en')]
        #combinations for every chosen analyzer with every chosen scoring function
        num_analyzers = len(analyzers)
        num_score_fun = len(score_functions)
        i=1
        sel_ana=['StemmingAnalyzer()','RegexAnalyzer()','FancyAnalyzer()','LanguageAnalyzer()']#text which will be used for graph and for mrr table
        scor_func=[' TF_IDF',' BM25F']
        for x in range(num_analyzers):
            for y in range(num_score_fun):
                print(sel_ana[x]+scor_func[y])
                # execute queries with the chosen configuration
                sr_1=executor(analyzers[x],score_functions[y],1) 
                #save results of the search engine
                sr_1.to_csv(os.getcwd()+"\part_1\part_1_1\Time_DATASET"+str(i)+".csv",index=False) 
                #compute the MRR 
                list_mrr1.append((sel_ana[x]+scor_func[y],MRR(sr_1,gt1))) 
                i+=1
        # save into a table with MRR evaluation for every search engine configuration 
        mrrs=pd.DataFrame(list_mrr1)
        mrrs.to_csv(os.getcwd()+"\part_1\part_1_1\Time_DATASET\mrr.csv", index=False) #store MRR table

In [ ]:
# exec the search engine with the different configurations for the Cranfield dataset
cranfield_ds = 0
search_engine(cranfield_ds) 

## CONTROLLARE PERCHE' NON LEGGE BENE IL FILE E CHECK DI TUTTA LA PARTE SUL TIME DATASET

In [31]:
# # exec the search engine with the different configurations for the Time dataset
# time_ds = 1
# search_engine(time_ds)

In [ ]:
sel_ana=['StemmingAnalyzer()','SimpleAnalyzer()','StandardAnalyzer()','RegexAnalyzer()','FancyAnalyzer()','NgramAnalyzer(5)','KeywordAnalyzer()','LanguageAnalyzer()']#text which will be used for graph and for mrr table
scor_func=['TF_IDF',' Frequency',' BM25F']